<a href="https://colab.research.google.com/github/subhra004/Subhra_04_mental_health_project/blob/main/Mental_Health_Support_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installed langchain_groq (for integrating Groq API with LangChain), langchain_core (core LangChain framework), and langchain_community (community-supported LangChain integrations) to build and enhance LLM-powered applications**

In [ ]:
!pip install langchain_groq langchain_core langchain_community

**Initialized a Groq LLM (llama-3.3-70b-versatile) with a temperature of 0 (for deterministic responses), invokes it with the query "Who is Lord Ram?", and prints the generated response**

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)


Lord Rama is a major deity in Hinduism and is considered one of the most revered and beloved figures in Indian mythology. He is the seventh avatar (incarnation) of the god Vishnu and is known for his exceptional virtues, bravery, and devotion to his family and duty.

According to the Hindu epic, the Ramayana, Lord Rama was born in the city of Ayodhya, in the kingdom of Kosala, to King Dasharatha and Queen Kausalya. He was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

Rama's life is a testament to his unwavering commitment to dharma (righteousness) and his unshakeable devotion to his family and community. Some of the key events and characteristics associated with Lord Rama include:

1. **Exile**: Rama was exiled to the forest for 14 years, along with his wife Sita and brother Lakshmana, due to a plot by his stepmother, Queen Kaikeyi.
2. **Sita's abduction**: During their exile, Sita was abducted by the demon king Ravana, who took her to his kingdom in Lanka

**Installed the pypdf library, which is used for reading, manipulating, and extracting text from PDF files in Python**

In [ ]:
!pip install pypdf

**Installed chromadb, a lightweight and efficient vector database used for storing and retrieving embeddings in AI/ML applications, particularly for retrieval-augmented generation (RAG) systems**

In [ ]:
!pip install chromadb

**Imported chromadb, retrieves the installation path of the chromadb package using os.path.dirname(chromadb.__file__), and prints the path to confirm its location**

In [ ]:
import chromadb
import os

# Get the path of the installed package
chroma_path = os.path.dirname(chromadb.__file__)
print("ChromaDB Path:", chroma_path)


ChromaDB Path: /usr/local/lib/python3.11/dist-packages/chromadb


**Installed the sentence_transformers library, which provides pre-trained models for generating high-quality text embeddings**

In [ ]:
!pip install sentence_transformers

**Built an AI-powered mental health chatbot using LangChain, Groq's llama-3.3-70b-versatile model, and ChromaDB, where it loads and processes a PDF (mental_health_Document.pdf), creates or loads a vector database with sentence-transformers embeddings, sets up a retrieval-augmented QA pipeline, and runs an interactive chatbot that retrieves context-aware responses to user queries**

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_groq import ChatGroq # Importing ChatGroq

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/mental_health_Document.pdf", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/usr/local/lib/python3.11/dist-packages/chromadb"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings) # Corrected indentation

  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........


<ipython-input-35-a309e20edbfe>:57: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-35-a309e20edbfe>:58: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings) # Corrected indentation



Human: I am so depressed 


<ipython-input-35-a309e20edbfe>:67: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot: I'm so sorry to hear that you're feeling depressed. It takes a lot of courage to acknowledge and share your emotions, and I'm here to listen and support you without judgment. Depression can be a really tough and isolating experience, but please know that you're not alone. I'm here to offer a supportive and non-judgmental space for you to express yourself.

Can you tell me more about what's been going on and how you've been feeling lately? Sometimes talking about it can help clarify things and give us a better understanding of what you're going through. I'm here to listen and offer support in any way I can. Remember, you don't have to face this alone, and there is help available. Would you like to talk about what's been on your mind, or is there something specific that's been bothering you?

Human: My boyfriend broke up with me
Chatbot: I'm so sorry to hear that you're going through this. Breakups can be incredibly painful and overwhelming. It's completely normal to feel sad, a

**Installed the Gradio library, which is used to create interactive web-based UIs for machine learning models and applications**

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 66.0 MB/s eta 0:00:00


1. **Loaded Mental Health Documents:** Uses PyPDFLoader and DirectoryLoader to load a mental health PDF, then splits the text into chunks for processing.

2. **Embedded and Stores Data:** Converts text chunks into embeddings using HuggingFaceBgeEmbeddings and stores them in ChromaDB for retrieval.

3. **Initialized LLM:** Connects to Groq’s Llama 3.3-70B model to generate responses.

4. **Retrieved Contextual Information:** Uses Retrieval-Augmented Generation (RAG) with LangChain’s RetrievalQA to provide context-aware responses.

5. **Created a Chatbot UI: **Uses Gradio to build an interactive chat interface where users can ask mental health-related questions.

6. **Handled Chat History:** Maintains conversation history and responds thoughtfully based on retrieved information.

7. **Launched the Web App:** The chatbot runs as a web application using Gradio Blocks, allowing users to interact with it in a browser.

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/mental_health_Document.pdf", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "/usr/local/lib/python3.11/dist-packages/chromadb"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history = []):
  if not user_input.strip():
    return "Please provide a valid input", history
  response = qa_chain.run(user_input)
  history.append((user_input, response))
  return "", history

with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
    gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

    chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

    gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

app.launch()

Intializing Chatbot.........


theme_schema%401.2.1.json:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69052ed9e16866fef1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
